In [5]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm as tqdm

In [6]:
# load all nih files into one big dataframe
#collect the following columns, where file id is the last 3 digits before the .csv 
#    'Sensor Data ID', 'file_id' 
#    'Yaw (rad)', 'Pitch (rad)', 'Roll (rad)', 'Rotation Rate X (rad/s)',
#    'Rotation Rate Y (rad/s)', 'Rotation Rate Z (rad/s)',
#    'User Acceleration X (m/s^2)', 'User Acceleration Y (m/s^2)',
#    'User Acceleration Z (m/s^2)', 'Altitude (m)', 'Speed (m/s)'
for file in tqdm(os.listdir('../data/nih/')):
    if file.endswith('.csv'):
        df = pd.read_csv('../data/nih/' + file)
        df['file_id'] = file.split('.')[0][-3:]
        df = df[['Sensor Data ID', 'file_id', 'Yaw (rad)', 'Pitch (rad)', 'Roll (rad)', 'Rotation Rate X (rad/s)',
                 'Rotation Rate Y (rad/s)', 'Rotation Rate Z (rad/s)', 'User Acceleration X (m/s^2)',
                 'User Acceleration Y (m/s^2)', 'User Acceleration Z (m/s^2)', 'Altitude (m)','Horizontal Accuracy (m)', 'Speed (m/s)']]
        if 'big_df' in locals():
            big_df = pd.concat([big_df, df])
        else:
            big_df = df


  0%|          | 0/40 [00:00<?, ?it/s]C:\Users\gabri\AppData\Local\Temp\ipykernel_16872\3068212805.py:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/nih/' + file)
  0%|          | 0/40 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [63]:
# load nih_train.npy and nih_val.npy and nih_test.npy to get the file ids
train_nih = np.load('../data/processed_datasets/nih_train.npy', allow_pickle=True)
val_nih = np.load('../data/processed_datasets/nih_val.npy', allow_pickle=True)
test_nih = np.load('../data/processed_datasets/nih_test.npy', allow_pickle=True)

train_ids  = np.unique(train_nih[:, :, -3])
val_ids = np.unique(val_nih[:, :, -3])
test_ids = np.unique(test_nih[:, :, -3])

# create train/val/test dataframes
train_df = big_df[big_df['file_id'].isin(train_ids)]
val_df = big_df[big_df['file_id'].isin(val_ids)]
test_df = big_df[big_df['file_id'].isin(test_ids)]

#drop na
train_df = train_df.dropna()
val_df = val_df.dropna()
test_df = test_df.dropna()

# reset index
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)

# save train/val/test dataframes
train_df.to_parquet('../data/processed_datasets/nih_ssl_train.parquet', index=False)
val_df.to_parquet('../data/processed_datasets/nih_ssl_val.parquet', index=False)
test_df.to_parquet('../data/processed_datasets/nih_ssl_test.parquet', index=False)

In [64]:
# find start indices, which are determined by looking at windows of length 128 with sliding window of legth 64 in unique file_ids (column 1), store 
start_indices = []
#create temporary column to store the index of each row
for file_id in tqdm(train_df['file_id'].unique()):
    file_data = train_df[train_df['file_id'] == file_id]
    start_index = file_data.index[0]

    for i in range(0, len(file_data), 64):
        if i+128 <= len(file_data):
            try:
                start_indices.append(i+start_index)
            except:
                print(i+start_index)


100%|██████████| 28/28 [00:35<00:00,  1.28s/it]


In [67]:
start_indices[-1]

25287085

In [30]:
train_df = pd.read_parquet('../data/processed_datasets/nih_ssl_train.parquet')

In [41]:
file_df = train_df[train_df['file_id'] == '143']
# get non reset index of first row
file_df.index[0]

23874295